# In this note book I am implementing parts of speech tagging for a text_file.

In [4]:
# Importing packages
from utils_pos import get_word_tag, preprocess  
import pandas as pd
from collections import defaultdict
import math
import numpy as np

Here we are using two tagged data set that are collect from  **the Wall Street Journal (WSJ)**.

1. One data set (WSJ-2_21.pos) will be used for training.
2. The other (WSJ-24.pos) for testing..

In [6]:
# load in the training corpus
with open("WSJ_02-21.pos", 'r') as f:
    training_corpus = f.readlines()

print(f"A few items of the training corpus list")
print(training_corpus[0:10])

A few items of the training corpus list
['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n', 'of\tIN\n', '``\t``\n', 'The\tDT\n', 'Misanthrope\tNN\n', "''\t''\n"]


In [7]:
# read the vocabulary data, split by each line of text, and save the list
with open("hmm_vocab.txt", 'r') as f:
    voc_l = f.read().split('\n')

print("First few items of the vocabulary list")
print(voc_l[0:50])
print()
print("A few items at the end of the vocabulary list")
print(voc_l[-50:])

First few items of the vocabulary list
['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m", "'n'", "'re", "'s", "'til", "'ve", '(', ')', ',', '-', '--', '--n--', '--unk--', '--unk_adj--', '--unk_adv--', '--unk_digit--', '--unk_noun--', '--unk_punct--', '--unk_upper--', '--unk_verb--', '.', '...', '0.01', '0.0108', '0.02', '0.03', '0.05', '0.1', '0.10', '0.12', '0.13', '0.15']

A few items at the end of the vocabulary list
['yards', 'yardstick', 'year', 'year-ago', 'year-before', 'year-earlier', 'year-end', 'year-on-year', 'year-round', 'year-to-date', 'year-to-year', 'yearlong', 'yearly', 'years', 'yeast', 'yelled', 'yelling', 'yellow', 'yen', 'yes', 'yesterday', 'yet', 'yield', 'yielded', 'yielding', 'yields', 'you', 'young', 'younger', 'youngest', 'youngsters', 'your', 'yourself', 'youth', 'youthful', 'yuppie', 'yuppies', 'zero', 'zero-coupon', 'zeroing', 'zeros', 'zinc', 'zip', 'zombie', 'zone', 'zones', 'zoning', 

In [8]:
# vocab: dictionary that has the index of the corresponding words
vocab = {} 

# Get the index of the corresponding words. 
for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i       
    
print("Vocabulary dictionary, key is the word, value is a unique integer")
cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 20:
        break

Vocabulary dictionary, key is the word, value is a unique integer
:0
!:1
#:2
$:3
%:4
&:5
':6
'':7
'40s:8
'60s:9
'70s:10
'80s:11
'86:12
'90s:13
'N:14
'S:15
'd:16
'em:17
'll:18
'm:19
'n':20


In [9]:
# load in the test corpus
with open("WSJ_24.pos", 'r') as f:
    y = f.readlines()

print("A sample of the test corpus")
print(y[0:10])

A sample of the test corpus
['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n', 'be\tVB\n', 'taken\tVBN\n', 'from\tIN\n', 'several\tJJ\n', 'vantage\tNN\n']


The test set (WSJ-24.pos) is read in to create y.

This contains both the test text and the true tag.
    
The test set has also been preprocessed to remove the tags to form **test_words.txt.**

preprocess function take  word from test.words and check it is in vocab or not .If it is in vocab then we store it other wise instead of word we store unknown tagged.

In [11]:
#corpus without tags, preprocessed
_, prep = preprocess(vocab, "test.words")     

print('The length of the preprocessed test corpus: ', len(prep))
print('This is a sample of  test_corpus: ')
print(prep[0:20])

The length of the preprocessed test corpus:  34199
This is a sample of  test_corpus: 
['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk--', 'points', 'this', 'week', ',', 'with', 'readings', 'on', 'trade', ',', 'output']


In [116]:
#create_dictionaries
def create_dictionaries(training_corpus, vocab) :
    
    # initialize the dictionaries using defaultdict
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    # Initialize "prev_tag" (previous tag) with the start state, denoted by '--s--'
    prev_tag = '--s--' 
    
    # use 'i' to track the line number in the corpus
    i = 0 
    
    # Each item in the training corpus contains a word and its POS tag
    # Go through each word and its tag in the training corpus
    for word_tag in training_corpus:
        
        # Increment the word_tag count
        i += 1
        
        # Every 50,000 words, print the word count
        if i % 50000 == 0:
            print(f"word count = {i}")
        # get the word and tag using the get_word_tag helper function (imported from utils_pos.py)
        word, tag = get_word_tag(word_tag,vocab) 
        
        # Increment the transition count for the previous word and tag
        transition_counts[(prev_tag, tag)] += 1
        
        # Increment the emission count for the tag and word
        emission_counts[(tag, word)] += 1

        # Increment the tag count
        tag_counts[tag] += 1

        # Set the previous tag to this tag (for the next iteration of the loop)
        prev_tag = tag
        
    return emission_counts, transition_counts, tag_counts

In [13]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

word count = 50000
word count = 100000
word count = 150000
word count = 200000
word count = 250000
word count = 300000
word count = 350000
word count = 400000
word count = 450000
word count = 500000
word count = 550000
word count = 600000
word count = 650000
word count = 700000
word count = 750000
word count = 800000
word count = 850000
word count = 900000
word count = 950000


In [14]:
len(training_corpus)

989860

In [15]:
# get all the POS states
states = sorted(tag_counts.keys())
print("Number of tags ", len(states))
print("Tags are followings : ",states)

Number of tags  46
Tags are followings :  ['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [16]:
#Show transition_counts, emission_count
print('Example of transition_counts\n')
for count in list(transition_counts.items())[:3] :
    print(count)
print()
print('Example of emission_count\n')
for em in list(emission_counts.items())[200:205] :
    print(em)

# A word which is called ambiguous if a word have many tag.
print("\nExample of ambiguous \n")
for tup,count in emission_counts.items() :
    if tup[1] == 'back' :
        print(tup, count)


Example of transition_counts

(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)

Example of emission_count

(('DT', 'any'), 721)
(('NN', 'decrease'), 7)
(('NN', 'insider-trading'), 5)
(('NNS', 'patterns'), 11)
(('MD', 'might'), 334)

Example of ambiguous 

('RB', 'back') 304
('VB', 'back') 20
('RP', 'back') 84
('JJ', 'back') 25
('NN', 'back') 29
('VBP', 'back') 4


# Assign a part of speech to a word and evaluate how well this approach work.
To assign a part of speech to a word, assign the most frequent POS for that word in the training set.
Then evaluate how well this approach works.
Each time you predict based on the most frequent POS for the given word, check whether the actual POS of that word is the same. If so, the prediction was correct!
Calculate the accuracy as the number of correct predictions divided by the total number of words for which you predicted the POS tag.

In [18]:
#implement predict_pos that computes the accuracy of your model
def predict_pos(prep, y, emission_counts, vocab, states):
    num_correct = 0  # initialize the number of correct prediction.
    all_words = set(emission_counts.keys()) # all_words contains the all distinct (tag, words) in the training set.
    total = len(y) # total is the number of distinct word in the test data set.
    for word, y_tup in zip(prep, y): 
        y_tup_l = y_tup.split() # Split the (word, POS) string into a list of two items.
        if len(y_tup_l) == 2:
            true_label = y_tup_l[1]
        else:
            continue
        
        count_final = 0
        pos_final = ''
        if word in vocab:
            for pos in states:
                key = (pos,word)
                if key in all_words:
                    count = emission_counts[key]
                    # For finding out maximum count of (tag, word) pair in the training data set
                    if count>count_final:
                        count_final = count
                        pos_final = pos       
        if pos_final == true_label:
                num_correct += 1
    accuracy = num_correct / total # accuracy calculation
    return accuracy

In [19]:
#Test the model
accuracy_predict_pos = predict_pos(prep, y, emission_counts, voc_l, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.8889


# Creatinig transition matrix

In [21]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    all_tags = sorted(tag_counts.keys()) # contains all distinct tags
    num_tags = len(all_tags) # number of distinct tags
    transition_mat = np.zeros((num_tags, num_tags)) # initialize transition matrix
    # Now creating transition matrix by using transition_counts dictionary
    for i in range(num_tags) :
        for j in range(num_tags) :
            key = (all_tags[i], all_tags[j]) # form the key for transition_counts. 
            count=0
            if key in transition_counts :
                count = transition_counts[key] # store corresponding transition_count value.
            count_prev_tag = tag_counts[all_tags[i]]
            transition_mat[i,j] = (count + alpha) / (count_prev_tag + alpha*num_tags) # Apply smoothing
    return transition_mat       
            

In [22]:
#Example of transition matrix
alpha = 0.001
for i in range(46):
    tag_counts.pop(i,None)

A = create_transition_matrix(alpha, tag_counts, transition_counts)
# Testing your function
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

#print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.000007040
A at row 3, col 1: 0.1691
              RBS            RP           SYM        TO            UH
RBS  2.217069e-06  2.217069e-06  2.217069e-06  0.008870  2.217069e-06
RP   3.756509e-07  7.516775e-04  3.756509e-07  0.051089  3.756509e-07
SYM  1.722772e-05  1.722772e-05  1.722772e-05  0.000017  1.722772e-05
TO   4.477336e-05  4.472863e-08  4.472863e-08  0.000090  4.477336e-05
UH   1.030439e-05  1.030439e-05  1.030439e-05  0.061837  3.092348e-02


# Creating emission matrix

In [24]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    num_tags = len(tag_counts) # number of tag in vocab
    all_tags = sorted(tag_counts.keys()) # store all distinct tag
    num_words = len(vocab) # store number of distinct words in the vocab
    emission_matrix = np.zeros((num_tags, num_words)) # initialize the emission_matrix
    emis_keys = set(list(emission_counts.keys())) # store set of all (tag,word) pairs.
    # Now creating emission matrix
    for i in range(num_tags):
        for j in range(num_words):
            count = 0
            key = (all_tags[i],vocab[j]) # form key for emission_counts
            if key in emission_counts.keys(): # check if key is in emission_count dictionary key or not
                count = emission_counts[key] # store corresponding key count
            count_tag = tag_counts[all_tags[i]]
            emission_matrix[i,j] = (count + alpha) / (count_tag+ alpha*num_words)# Doing smoothing
    return emission_matrix

In [114]:
# creating emission probability matrix.
for i in range(46):
    tag_counts.pop(i,None)

B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")

# Try viewing emissions for a few words in a sample dataframe
cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']

# Get the integer ID for each word
cols = [vocab[a] for a in cidx]

# Choose POS tags to show in a sample dataframe
rvals =['CD','NN','NNS', 'VB','RB','RP']

# For each POS tag, get the row number from the 'states' list
rows = [states.index(a) for a in rvals]

# Get the emissions for the sample of words, and the sample of POS tags
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07


# Viterbi Algorithm

# Initialize best_path and best_prob matrix

In [28]:
def initialize(states, tag_counts, A, B, corpus, vocab):
    num_tags = len(tag_counts) # store number of distinct tags
    best_probs = np.zeros((num_tags, len(corpus))) # initialize the matrix best_probs of order (num_tags X (number of distinct word in test data set))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)# initialize the matrix best_paths of order (num_tags X (number of distinct word in test data set))
    s_idx = states.index("--s--") # store the index of the starting tag.
    
    # Now initialize first column of the best_probs
    for i in range(num_tags):
        if A[s_idx,i] == 0:
            best_probs[i,0] = float('-inf')
        else:
            # vocab[corpus[0]] is the index of the first word of the corpus.
            best_probs[i,0] = math.log(A[s_idx,i]) + math.log(B[i,vocab[corpus[0]]] )
    return best_probs, best_paths

In [29]:
# Initializing best_probs and best_paths
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [30]:
# Test the function
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -22.6098
best_paths[2,3]: 0.0000


# Forward pass

In [66]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    num_tags = best_probs.shape[0]
    for i in range(1, len(test_corpus)):
        if i % 5000 == 0:
            print("Words processed: {:>8}".format(i))
        for j in range(num_tags):
            #initialization the variable
            best_prob_i =  float("-inf")
            best_path_i = None
            # we store the index of the previous tag k in the best_path in which ith word have jth tag with high probability and 
            # store the corresponding highest probability in the (j,i) th cell of best_prob.
            for k in range(num_tags):
                prob = best_probs[k,i-1]+math.log(A[k,j]) +math.log(B[j,vocab[test_corpus[i]]])
                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k
            best_probs[j,i] = best_prob_i
            best_paths[j,i] = best_path_i
    return best_probs, best_paths

In [33]:
# this will take a few minutes to run => processes ~ 30,000 words
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

Words processed:     5000
Words processed:    10000
Words processed:    15000
Words processed:    20000
Words processed:    25000
Words processed:    30000


In [34]:
# Test this function 
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}")

best_probs[0,1]: -24.7822
best_probs[0,4]: -49.5601


# Backward step

In [78]:
def viterbi_backward(best_probs, best_paths, corpus, states):
        # Get the number of words in the corpus
        # which is also the number of columns in best_probs, best_paths
        m = best_paths.shape[1]
        z = [None] * m # Initialize array z, same length as the corpus
        num_tags = best_probs.shape[0]
        best_prob_for_last_word = float('-inf')
        pred = [None] * m
        for k in range(num_tags):
            if best_probs[k,-1]>best_prob_for_last_word:
                best_prob_for_last_word = best_probs[k,-1]
                z[m - 1]=k
        pred[m - 1] = states[k]
        for i in range(len(corpus)-2, -1, -1):
            z[i] = best_paths[z[i+1], i+1]
            pred[i] = states[z[i]]
        return pred

In [80]:
#Example
pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']


In [82]:
# UNQ_C8 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: compute_accuracy
def compute_accuracy(pred, y):
    num_correct = 0 #initialize the number of correct prediction
    total = 0 
    for prediction, y in zip(pred, y):
        word_tag_tuple = y.split() # split in the form of list (word, tag)
        if len(word_tag_tuple)!=2:
            continue 
        word, tag = word_tag_tuple
        if prediction == tag: # if prediction id is correct increase the number of correct prediction by 1.
            num_correct += 1
        total += 1 # at the end of the loop total contains the number of total word in y.
    return (num_correct/total)

In [84]:
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

Accuracy of the Viterbi algorithm is 0.9531


In [95]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to C:\Users\Mrinmoy
[nltk_data]     Bera\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# parts_of_speech_tagging of a text file 
Input a text file following function return corresponding parts of speech tagging.

In [97]:
def parts_of_speech_tagging(text_file, states, tag_counts, A, B, vocab) :
    _, prep = preprocess(vocab, text_file)  
    
    # Initializing best_probs and best_paths
    best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

    #veterbi_forward step
    best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

    #veterbi_backward
    pred = viterbi_backward(best_probs, best_paths, prep, states)

    return pred

In [109]:
#prediction of corresponding text file
pred = parts_of_speech_tagging("test.words", states, tag_counts, A, B, vocab)

Words processed:     5000
Words processed:    10000
Words processed:    15000
Words processed:    20000
Words processed:    25000
Words processed:    30000


In [111]:
# Eample of parts_of_speech_tagging 
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']
